In [1]:
import numpy as np
import pandas as pd
from senticnet5 import senticnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from scipy import sparse
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_excel('hand8_k_random.xlsx')
print(len(df))

8501


In [3]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain
from skmultilearn.ensemble import RakelD
from skmultilearn.adapt import BRkNNaClassifier, BRkNNbClassifier

In [4]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
X = df['Text']
Y = df[['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']]

In [5]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Text'])
print(len(vectorizer.vocabulary_))

17402


In [6]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [7]:
def evaluation(score_list,predict_score_list):
    filter_corr = []
    exmatch = 0
    atleast1 = 0
    md1 = 0
    one_f = 0
    more_f = 0
    zero_f = 0
    sm = 0
    sdensity = 0
    hammval = 0
    test_len = len(predict_score_list[0])
    for j in range(test_len):
        cnt=0
        for i in range(8):
            hammval+=(score_list[i][j] ^ int(predict_score_list[i][j]))
            if(score_list[i][j]==1):
                cnt+=1
                sm+=1
        sdensity+=cnt/8
        if(cnt==0):
            zero_f+=1
        if(cnt==1):
            one_f+=1
        if(cnt>1):
            more_f+=1
        for i in range(8):
            mf = True
            if(int(predict_score_list[i][j])!=score_list[i][j]):
                mf=False
                break
        if(mf==True):
            exmatch+=1
            filter_corr.append(j)
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                atleast1+=1
                break
        mf = False
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                if(mf==True):
                    md1+=1
                    filter_corr.append(j)
                    break
                mf=True
    #print("Label Cardinality: "+ str(sm/test_len))
    #print("Label Density: "+ str(sdensity/test_len))
    print("Hamming Loss: "+str(hammval/(test_len*8)))
    hamlos = hammval/(test_len*8)
    print("Exact Prediction: "+str(exmatch/test_len))
    sub_accu = exmatch/test_len
    #print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
    #print("More than one label predicted: "+str(md1/more_f))
    tp_sum = 0
    fp_sum = 0
    fn_sum = 0
    macro_preci = 0
    macro_recall = 0
    macro_f1 = 0
    for i in range(len(score_list)):
        tmp = confusion_matrix(score_list[i],predict_score_list[i])
        tp_sum+=tmp[0][0]
        fp_sum+=tmp[0][1]
        fn_sum+=tmp[1][0]
        macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
        macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
        macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
        macro_preci+=macro_preci_tmp
        macro_recall+=macro_recall_tmp
        #print(macro_f1)
    micro_preci = tp_sum/(tp_sum+fp_sum)
    micro_recall = tp_sum/(tp_sum+fn_sum)
    micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
    macro_preci/=8
    macro_recall/=8
    macro_f1/=8
    #print(micro_preci,micro_recall,micro_f1)
    #print(macro_preci,macro_recall,macro_f1)
    print("Macro F-Score: "+str(macro_f1))
    print("Micro F-Score: "+str(micro_f1))
    col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
    tmp = 0
    for i in range(len(score_list)):
        score = accuracy_score(score_list[i],predict_score_list[i]) 
        #print(col_names[i]+" accuracy: "+str(score))
        tmp += score
    print("Average Accuracy: " + str(tmp/8))
    avg_accu = tmp/8
    return (hamlos,sub_accu,macro_f1,micro_f1,avg_accu)

In [8]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].values.tolist()),np.array(Y.iloc[test_index].values.tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    classifier = BRkNNaClassifier(k=100)
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)
    
    y_pred_val = y_pred.toarray()
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val.T.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650, 8) (851, 8)


C:\Users\Rainmaker\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_neighbors=100 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Hamming Loss: 0.16994712103407755
Exact Prediction: 0.172737955346651
Macro F-Score: 0.8830562098238796
Micro F-Score: 0.9025848278184727
Average Accuracy: 0.8300528789659225


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.17955882352941177
Exact Prediction: 0.1811764705882353
Macro F-Score: 0.8783008294170755
Micro F-Score: 0.8968139947604157
Average Accuracy: 0.8204411764705882


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.17647058823529413
Exact Prediction: 0.1635294117647059
Macro F-Score: 0.88111763263869
Micro F-Score: 0.8983050847457628
Average Accuracy: 0.8235294117647058


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.17161764705882354
Exact Prediction: 0.18823529411764706
Macro F-Score: 0.8843501402288436
Micro F-Score: 0.9013775035916505
Average Accuracy: 0.8283823529411765


k_fold validation: 5
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.170294

In [9]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [10]:
df_res = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_res

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.169947,0.172738,0.883056,0.902585,0.830053
1,2,0.179559,0.181176,0.878301,0.896814,0.820441
2,3,0.176471,0.163529,0.881118,0.898305,0.823529
3,4,0.171618,0.188235,0.884350,0.901378,0.828382
4,5,0.170294,0.192941,0.884805,0.902146,0.829706
5,6,0.170441,0.182353,0.886045,0.902400,0.829559
6,7,0.169265,0.177647,0.886822,0.902532,0.830735
7,8,0.171029,0.194118,0.886132,0.901898,0.828971
8,9,0.173971,0.204706,0.881849,0.899788,0.826029
9,10,0.169853,0.172941,0.887030,0.902458,0.830147


In [11]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].values.tolist()),np.array(Y.iloc[test_index].values.tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    classifier = BRkNNbClassifier(k=100)
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)
    
    y_pred_val = y_pred.toarray()
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val.T.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650, 8) (851, 8)


C:\Users\Rainmaker\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_neighbors=100 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Hamming Loss: 0.2842244418331375
Exact Prediction: 0.06580493537015276
Macro F-Score: 0.7889418879930359
Micro F-Score: 0.8233360723089564
Average Accuracy: 0.7157755581668624


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.27794117647058825
Exact Prediction: 0.06823529411764706
Macro F-Score: 0.7962738163967259
Micro F-Score: 0.827018121911038
Average Accuracy: 0.7220588235294116


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.2889705882352941
Exact Prediction: 0.06235294117647059
Macro F-Score: 0.7856884391578026
Micro F-Score: 0.8190441108757712
Average Accuracy: 0.7110294117647059


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.28794117647058826
Exact Prediction: 0.05647058823529412
Macro F-Score: 0.7870300547128514
Micro F-Score: 0.8203669724770642
Average Accuracy: 0.7120588235294117


k_fold validation: 5
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.281

In [12]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [13]:
df_res = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_res

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.284224,0.065805,0.788942,0.823336,0.715776
1,2,0.277941,0.068235,0.796274,0.827018,0.722059
2,3,0.288971,0.062353,0.785688,0.819044,0.711029
3,4,0.287941,0.056471,0.787030,0.820367,0.712059
4,5,0.281765,0.062353,0.791513,0.824638,0.718235
5,6,0.289118,0.057647,0.789241,0.820161,0.710882
6,7,0.282794,0.062353,0.791424,0.823691,0.717206
7,8,0.285294,0.060000,0.791943,0.822960,0.714706
8,9,0.280294,0.076471,0.793074,0.824816,0.719706
9,10,0.283676,0.061176,0.789935,0.823011,0.716324
